In [1]:
import re
import sys
import json
import gzip
import logging
import pickle as pkl
from functools import lru_cache
from multiprocessing import Pool
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import itertools
from annoy import AnnoyIndex

import pymorphy2
import numpy as np
import pandas as pd
from annoy import AnnoyIndex
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import word_tokenize
from application.machine.Seeker import Seeker

MORPH = pymorphy2.MorphAnalyzer()
porter = PorterStemmer()

In [2]:
data = pd.read_csv('scopus.csv')

In [3]:
data.head()

,Авторы,Идентификатор автора(ов),Название,Год,Название источника,Том,Выпуск,Статья №,Страница начала,Страница окончания,...,Цитирования,DOI,Ссылка,Краткое описание,Ключевые слова автора,Тип документа,Стадия публикации,Тип доступа,Источник,EID
0,"Chow A.H.F., Li S.",36146863600;56719990800;,Modelling and managing bus service regularity ...,2019,Transportmetrica B,7,1,NaN,82.0,106.0,...,1.0,10.1080/21680566.2017.1353450,https://www.scopus.com/inward/record.uri?eid=2...,This paper presents a multi-modal framework fo...,Bus bunching; multi-modal management; transit ...,Article,Final,NaN,Scopus,2-s2.0-85024501234
1,"Ortigosa J., Gayah V.V., Menendez M.",55863721200;16244850200;7102690501;,Analysis of one-way and two-way street configu...,2019,Transportmetrica B,7,1,NaN,61.0,81.0,...,NaN,10.1080/21680566.2017.1337528,https://www.scopus.com/inward/record.uri?eid=2...,"In this paper, we study traffic behavior on th...",DTA; grid networks; left turns; one-way street...,Article,Final,NaN,Scopus,2-s2.0-85021050453
2,"Sau J., Monteil J., Bouroche M.",55490778800;56085947000;15134891000;,State-space linear stability analysis of plato...,2019,Transportmetrica B,7,1,NaN,18.0,43.0,...,NaN,10.1080/21680566.2017.1308846,https://www.scopus.com/inward/record.uri?eid=2...,Platoons of vehicles relying on inter-vehicle ...,car-following models; cooperative systems; lin...,Article,Final,NaN,Scopus,2-s2.0-85017096656
3,"Wandelt S., Sun X., Zhang J.",23468068100;55806545400;57188731724;,Evolution of domestic airport networks: a revi...,2019,Transportmetrica B,7,1,NaN,1.0,17.0,...,2.0,10.1080/21680566.2017.1301274,https://www.scopus.com/inward/record.uri?eid=2...,Previous studies on airport networks are stron...,complex network; domestic airport network; Net...,Article,Final,NaN,Scopus,2-s2.0-85015645443
4,"Singh M.K., Pandey G., Ramachandra Rao K.",57205283841;56482819900;55549024500;,Optimization of work zone segments on urban ro...,2019,Periodica Polytechnica Transportation Engineering,47,1,NaN,29.0,35.0,...,NaN,10.3311/PPtr.9992,https://www.scopus.com/inward/record.uri?eid=2...,Increased delays and reduced speeds in work zo...,Calibration; Cellular automata; Delay; Heterog...,Article,Final,Open Access,Scopus,2-s2.0-85059378072


In [4]:
stop_words = list(stopwords.words('english'))

In [5]:
columns = ['Авторы', 'Название', 'Название источника', 'Ссылка', 'Краткое описание', 'Ключевые слова автора']
delete_columns = [column not in columns for column in data.columns]
data = data.drop(columns = data.columns[delete_columns])

In [6]:
data.head()

,Авторы,Название,Название источника,Ссылка,Краткое описание,Ключевые слова автора
0,"Chow A.H.F., Li S.",Modelling and managing bus service regularity ...,Transportmetrica B,https://www.scopus.com/inward/record.uri?eid=2...,This paper presents a multi-modal framework fo...,Bus bunching; multi-modal management; transit ...
1,"Ortigosa J., Gayah V.V., Menendez M.",Analysis of one-way and two-way street configu...,Transportmetrica B,https://www.scopus.com/inward/record.uri?eid=2...,"In this paper, we study traffic behavior on th...",DTA; grid networks; left turns; one-way street...
2,"Sau J., Monteil J., Bouroche M.",State-space linear stability analysis of plato...,Transportmetrica B,https://www.scopus.com/inward/record.uri?eid=2...,Platoons of vehicles relying on inter-vehicle ...,car-following models; cooperative systems; lin...
3,"Wandelt S., Sun X., Zhang J.",Evolution of domestic airport networks: a revi...,Transportmetrica B,https://www.scopus.com/inward/record.uri?eid=2...,Previous studies on airport networks are stron...,complex network; domestic airport network; Net...
4,"Singh M.K., Pandey G., Ramachandra Rao K.",Optimization of work zone segments on urban ro...,Periodica Polytechnica Transportation Engineering,https://www.scopus.com/inward/record.uri?eid=2...,Increased delays and reduced speeds in work zo...,Calibration; Cellular automata; Delay; Heterog...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 6 columns):
Авторы                   4324 non-null object
Название                 4324 non-null object
Название источника       4324 non-null object
Ссылка                   4324 non-null object
Краткое описание         4324 non-null object
Ключевые слова автора    2080 non-null object
dtypes: object(6)
memory usage: 202.8+ KB


In [8]:
data = data.fillna(" ")

In [9]:
data.head()['Краткое описание'][0]

'This paper presents a multi-modal framework for modelling and analysing different headway control strategies for improving bus service regularity. The interaction between buses and their surrounding traffic is captured in a multi-modal system through a Hamilton–Jacobi formulation of kinematic wave model. This paper further presents a set of signal-based strategies which regulate bus headway through adjusting signal timing plans. The results reveal that the capability of regulating bus headway disturbances through utilising traffic signals is important for maximising bus service regularity and coordinating the bus dynamics with surrounding traffic. This study generates new insights on managing bus reliability and multi-modal traffic in busy urban networks. © 2019, © 2019 Hong Kong Society for Transportation Studies Limited.'

In [10]:
from gensim.summarization import summarize

In [11]:
@lru_cache(maxsize=100000)
def get_normal_form (i):
    return porter.stem(i.lower())

def normalize_text(text):
    #text =summarize(text)
    text = text[:text.find('©')]
    tokens = word_tokenize(text)
    text = [word for word in tokens if word.isalpha()]
    normalized = [get_normal_form(word) for word in text]
    return ' '.join([word for word in normalized if word not in stop_words])

def normalize_description(text):
    try:
        text = summarize(text, ratio=0.5)
    except ValueError:
        pass
    return normalize_text(text)

def normalize_keys(text):
    words = []
    for word in text.split(';'):
        words.append((word.lower()).replace(' ', ''))
    return ' '.join([word for word in words])

In [12]:
data['Title'] = data['Название'].apply(lambda x: normalize_text(x))
data['Description'] = data['Краткое описание'].apply(lambda x: normalize_description(x))
data['Key words'] = data['Ключевые слова автора'].apply(lambda x: normalize_keys(x))

In [13]:
sentences = (data['Title'] + ' ' + data['Description'] + data['Key words']).str.split()
keys = (data['Key words']).str.split()

In [14]:
key_words = []
for listik in keys.values:
    for word in listik:
        if word not in key_words:
            key_words.append(word)

In [15]:
model = Word2Vec(sentences, size=100, workers=2, iter=14)
model.save('./w2v_products1.w2v_gensim')

In [16]:
model.similar_by_vector(model['decreas'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('decreas', 1.0),
 ('increas', 0.9257568120956421),
 ('elev', 0.8702961802482605),
 ('greater', 0.8368594646453857),
 ('cycl', 0.8352645039558411),
 ('warm', 0.8312910199165344),
 ('permafrost', 0.8303860425949097),
 ('concentr', 0.8247277140617371),
 ('braze', 0.8241923451423645),
 ('significantli', 0.8180956840515137)]

In [17]:
model.similar_by_vector(model['decreas'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('decreas', 1.0),
 ('increas', 0.9257568120956421),
 ('elev', 0.8702961802482605),
 ('greater', 0.8368594646453857),
 ('cycl', 0.8352645039558411),
 ('warm', 0.8312910199165344),
 ('permafrost', 0.8303860425949097),
 ('concentr', 0.8247277140617371),
 ('braze', 0.8241923451423645),
 ('significantli', 0.8180956840515137)]

In [18]:
model.similar_by_vector(model['equat'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('equat', 1.0000001192092896),
 ('pde', 0.9223195910453796),
 ('equatio', 0.9091947674751282),
 ('vri', 0.906840443611145),
 ('nonlinear', 0.899742066860199),
 ('ordinari', 0.8970274925231934),
 ('exactsolution', 0.8936541676521301),
 ('painlevé', 0.8907214403152466),
 ('travel', 0.8893516659736633),
 ('calculu', 0.8882980346679688)]

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

titles_ = (data['Title'] + ' ' + data['Description'] + ' ' +  data['Key words']).values
#titles_ = (data['Key words']).values

tfidf = TfidfVectorizer(ngram_range=(1,1))
tfidf.fit(titles_)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [20]:
tf_idf_vocab = {}
for i in model.wv.vocab.keys():
    if i in tfidf.vocabulary_:
        if i in key_words:
            tf_idf_vocab[i] = tfidf.idf_[tfidf.vocabulary_[i]] * 24
        else:  
            tf_idf_vocab[i] = tfidf.idf_[tfidf.vocabulary_[i]]

tf_idf_vocab = {}
for i in model.wv.vocab.keys():
    if i in tfidf.vocabulary_:
        tf_idf_vec = tfidf.transform([i])
        tf_idf_vocab[i] = tf_idf_vec[:, tfidf.vocabulary_[i]].toarray().flatten()[0]

In [21]:
import tqdm
data_storage = {i[0]:i[1]['Title']+ ' ' + i[1]['Description']  + ' ' +  i[1][
        'Key words'] for i in data.iterrows()}
#data_storage = {i[0]:i[1]['Ключевые слова автора'] for i in data.iterrows()}
data_storage_norm = {}
for i in tqdm.tqdm(data_storage):
    text = data_storage[i]
    vec = np.zeros(100)
    for word in text.split(' '):
        if word in model and word in tf_idf_vocab:
            vec+=model[word] * tf_idf_vocab[word]

    data_storage_norm[i] = vec

  0%|                                                                                         | 0/4324 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
100%|█████████████████████████████████████████████████████████████████████████████| 4324/4324 [00:05<00:00, 745.46it/s]


In [22]:
from annoy import AnnoyIndex

NUM_TREES = 15
VEC_SIZE_EMB = 100

counter = 0
map_id_2_prod_hash = {}
index_title_emb = AnnoyIndex(VEC_SIZE_EMB)

print("Build annoy base")
for prod_hash in data_storage_norm:
    title_vec = data_storage_norm[prod_hash] # Вытаскиваем вектор
    
    index_title_emb.add_item(counter, title_vec) # Кладем в анной
    
    map_id_2_prod_hash[counter] = prod_hash # сохраняем мапу - (id в анное -> продукт_id)
    
    counter += 1
    if counter % 10000 == 1:
        print("computed for %d" % counter)
        
index_title_emb.build(NUM_TREES)
print("builded")

Build annoy base
computed for 1
builded


In [23]:
index_title_emb.save('./annoy_checkbox')
pkl.dump(map_id_2_prod_hash, open('map_id_to_hash_products_checkbox.dict', 'wb'))

In [24]:
pd.DataFrame(data_storage_norm).to_csv(
        'data_storage_norm_checkbox.csv', index=None)

In [30]:
data_storage_norm = pd.read_csv('data_storage_norm_checkbox.csv')

In [31]:


title_id = list(data_storage_norm.keys())[np.random.randint(2000)]
title_id = 2094
print('Запрос', data_storage[title_id])

annoy_res = list(index_title_emb.get_nns_by_vector(data_storage_norm.iloc[:,title_id], 13, include_distances=True))

print('\n\nСоседи:')

for annoy_id, annoy_sim in itertools.islice(zip(*annoy_res), 13):
    image_id = map_id_2_prod_hash[annoy_id]
    print('Название: ' + data.iloc[image_id]['Название'] +'\n' +  data.iloc[image_id]['Краткое описание'], 1 - annoy_sim ** 2 / 2)
    print('Ключевые слова: ' + data.iloc[image_id]['Ключевые слова автора'])
    print('___' * 13)

Запрос painlevé analysi exact solut equat descript nonlinear wave painlevé test appli investig integr equat show equat integr exact solut equat exist analyt solut equat found exactsolution kuramoto-sivashinskyequation logisticfunction nonlineardifferentialequation nonlinearwave painlevéproperty painlevétest


Соседи:
Painlevé analysis and exact solutions of the fourth-order equation for description of nonlinear waves
The fourth-order equation for description of nonlinear waves is considered. A few variants of this equation are studied. Painlevé test is applied to investigate integrability of these equations. We show that all these equations are not integrable, but some exact solutions of these equations exist. Analytic solutions in closed-form of the equations are found. © 2015 Elsevier B.V. 1.0
Exact solution; Kuramoto-Sivashinsky equation; Logistic function; Nonlinear differential equation; Nonlinear wave; Painlevé property; Painlevé test
_______________________________________
Painl

The integrability of the traveling wave reduction for the Triki–Biswas equation is considered. Two first integrals for this equation are found. It is shown that the Triki–Biswas equation is transformed to the nonlinear ordinary differential equation of the first order. The general solution of the traveling wave reduction for the Triki–Biswas equation is written using the quadrature. The general solution of the traveling wave reduction for the Kaup–Newell equation is given. Special solutions of the Triki–Biswas equation are presented. © 2019 Elsevier GmbH 0.9952323829049259
Exact solution; General solution; Integrability; Traveling wave; Triki–Biswas equation
_______________________________________
A note on New Exact solutions for the Kawahara equation using Exp-function method
Exact solutions of the Kawahara equation by Assas [L.M.B. Assas, New Exact solutions for the Kawahara equation using Exp-function method, J. Comput. Appl. Math. 233 (2009) 97102] are analyzed. It is shown that a

In [27]:
filename_key_words = '_checkbox'
map_id_2_prod_hash = pkl.load(
            open('map_id_to_hash_products' + filename_key_words + '.dict',
                 'rb'))

index_title_emb = AnnoyIndex(100)
index_title_emb.load('./annoy' + filename_key_words)
model = Word2Vec.load(
    './w2v_products.w2v_gensim')

In [28]:


vec = np.zeros(100)
for i in listik:
    part_of_vec = None
    try:
        part_of_vec = model[i]
    except KeyError:
        pass
    if part_of_vec is not None:
        vec += part_of_vec

annoy_res = list(
    index_title_emb.get_nns_by_vector(vec, 13, include_distances=True))

In [29]:
print('\n\nСоседи:')

for annoy_id, annoy_sim in itertools.islice(zip(*annoy_res), 13):
    image_id = map_id_2_prod_hash[annoy_id]
    print(data.iloc[image_id]['Название'] + data.iloc[image_id]['Краткое описание'], 1 - annoy_sim ** 2 / 2)
    print(data.iloc[image_id]['Ключевые слова автора'])
    print('___' * 13)



Соседи:
Existence results for a fraction hybrid differential inclusion with Caputo–Hadamard type fractional derivativeIn this manuscript, we talk over the existence of solutions of a class of hybrid Caputo–Hadamard fractional differential inclusions with Dirichlet boundary conditions. Our results are based on the Arzelá–Ascoli theorem and some suitable theorems of fixed point theory. As well, to illustrate our results, we confront the exceptional case of the fractional differential inclusions with examples. © 2019, The Author(s). 3.422854177870249e-08
Caputo–Hadamard fractional derivative; Fractional differential inclusion; Hybrid boundary value problem
_______________________________________
Multiple radial solutions for Dirichlet problem involving two mean curvature equations in Euclidean and Minkowski spacesIn this paper, we establish the existence of multiple positive radial solutions for the Dirichlet problem involving two mean curvature equations of spacelike graphs in Euclidea

_______________________________________
A connection between weighted Hardy’s inequality and half-linear dynamic equationsIn this paper, we give an affirmative answer to the following question: Is the solvability of some nonlinear dynamic equations on a time scale T not only sufficient but in a certain sense also necessary for the validity of some dynamic Hardy-type inequalities with two different weights? In fact, this answer will give a new characterization of the weights in a weighted Hardy-type inequality on time scales. The results contain the results when T= R, T= N, and when T=qN0 as special cases. Some applications are given for illustrations. © 2019, The Author(s). 3.422854177870249e-08
Dynamic equations; Hardy’s inequality; Time scales
_______________________________________
